In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:

def get_stock_data(tickers, start_date, end_date):
    """Fetch stock price and volume data from Yahoo Finance"""
    all_data = {'Close': {}, 'Volume': {}}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            df = stock.history(start=start_date, end=end_date)
            if not df.empty:
                all_data['Close'][ticker] = df['Close']
                all_data['Volume'][ticker] = df['Volume']
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
    return pd.DataFrame(all_data['Close']), pd.DataFrame(all_data['Volume'])

def get_sector_info(tickers):
    """Fetch sector information for each ticker"""
    sector_data = {}
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            sector = info.get('sector', 'Unknown')
            sector_data[ticker] = sector
        except Exception as e:
            print(f"Error fetching sector for {ticker}: {e}")
            sector_data[ticker] = 'Unknown'
    return sector_data

def calculate_rs_and_momentum(tickers, period_days=252):
    """
    Calculate RS Rankings and Momentum (Trend Template) criteria with volume confirmation
    Default period is 252 days (approx 1 trading year)
    """
    # Set date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=period_days + 60)  # Extra 60 days for 200-day SMA trend
    
    # Get S&P 500 data as benchmark (^GSPC)
    sp500 = yf.Ticker("^GSPC").history(start=start_date, end=end_date)
    if sp500.empty:
        raise ValueError("No S&P 500 data available")
    
    # Calculate total return for S&P 500 over the main period (last 252 days)
    sp500_main_period = sp500[-period_days:]
    sp500_start = sp500_main_period['Close'].iloc[0]
    sp500_end = sp500_main_period['Close'].iloc[-1]
    sp500_return = (sp500_end - sp500_start) / sp500_start
    
    # Get stock data (prices and volumes)
    close_data, volume_data = get_stock_data(tickers, start_date, end_date)
    if close_data.empty:
        raise ValueError("No stock data retrieved")
    
    # Get sector data
    sector_data = get_sector_info(tickers)
    
    # Initialize results dictionary with volume criterion
    results = {
        'Ticker': [],
        'Sector': [],
        'RS_Value': [],
        'Price': [],
        '50_SMA': [],
        '150_SMA': [],
        '200_SMA': [],
        '52W_High': [],
        'Price_Above_150_SMA': [],
        'Price_Above_200_SMA': [],
        '150_SMA_Above_200_SMA': [],
        '50_SMA_Above_150_SMA': [],
        'Within_25pct_52W_High': [],
        'Meets_Trend_Template': [],
        'Volume_Spike_40pct': []  # New column for volume confirmation
    }
    
    # Process each ticker
    for ticker in tickers:
        results['Ticker'].append(ticker)
        results['Sector'].append(sector_data.get(ticker, 'Unknown'))
        
        df_close = close_data.get(ticker, pd.Series()).dropna()
        df_volume = volume_data.get(ticker, pd.Series()).dropna()
        
        if len(df_close) < 200:  # Need at least 200 days for SMA calculations
            print(f"Insufficient data for {ticker} (< 200 days)")
            results['RS_Value'].append(np.nan)
            results['Price'].append(np.nan)
            results['50_SMA'].append(np.nan)
            results['150_SMA'].append(np.nan)
            results['200_SMA'].append(np.nan)
            results['52W_High'].append(np.nan)
            results['Price_Above_150_SMA'].append(False)
            results['Price_Above_200_SMA'].append(False)
            results['150_SMA_Above_200_SMA'].append(False)
            results['50_SMA_Above_150_SMA'].append(False)
            results['Within_25pct_52W_High'].append(False)
            results['Meets_Trend_Template'].append(False)
            results['Volume_Spike_40pct'].append(False)
            continue
        
        try:
            # Current price and volume
            current_price = df_close.iloc[-1]
            current_volume = df_volume.iloc[-1] if not df_volume.empty else np.nan
            
            # Calculate moving averages
            sma_50 = df_close.rolling(window=50).mean().iloc[-1]
            sma_150 = df_close.rolling(window=150).mean().iloc[-1]
            sma_200 = df_close.rolling(window=200).mean().iloc[-1]
            
            # Check 200-day SMA trend (up for at least 20 days ~ 1 month)
            sma_200_trend = df_close.rolling(window=200).mean()
            sma_200_month_ago = sma_200_trend.iloc[-21]  # Roughly 1 month back
            
            # 52-week high (last 252 days)
            year_high = df_close[-252:].max()
            
            # Calculate RS Value (last 252 days)
            stock_start = df_close[-252:].iloc[0]
            stock_end = df_close[-252:].iloc[-1]
            stock_return = (stock_end - stock_start) / stock_start
            rs_value = stock_return / sp500_return if sp500_return != 0 else np.nan
            
            # Individual Trend Template Components
            price_above_150_sma = current_price > sma_150
            price_above_200_sma = current_price > sma_200
            sma_150_above_200 = sma_150 > sma_200
            sma_50_above_150 = sma_50 > sma_150
            within_25pct_52w_high = current_price >= year_high * 0.75
            
            # Overall Trend Template
            meets_trend_template = (
                price_above_150_sma and
                price_above_200_sma and
                sma_150_above_200 and
                sma_50_above_150 and
                within_25pct_52w_high
            )
            
            # Volume Confirmation (40% above 50-day average)
            avg_volume_50 = df_volume.rolling(window=50).mean().iloc[-1] if not df_volume.empty else np.nan
            volume_spike_40pct = (current_volume / avg_volume_50) >= 1.4 if pd.notna(avg_volume_50) and avg_volume_50 != 0 else False
            
            # Append results
            results['RS_Value'].append(rs_value)
            results['Price'].append(current_price)
            results['50_SMA'].append(sma_50)
            results['150_SMA'].append(sma_150)
            results['200_SMA'].append(sma_200)
            results['52W_High'].append(year_high)
            results['Price_Above_150_SMA'].append(price_above_150_sma)
            results['Price_Above_200_SMA'].append(price_above_200_sma)
            results['150_SMA_Above_200_SMA'].append(sma_150_above_200)
            results['50_SMA_Above_150_SMA'].append(sma_50_above_150)
            results['Within_25pct_52W_High'].append(within_25pct_52w_high)
            results['Meets_Trend_Template'].append(meets_trend_template)
            results['Volume_Spike_40pct'].append(volume_spike_40pct)
            
        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            results['RS_Value'].append(np.nan)
            results['Price'].append(np.nan)
            results['50_SMA'].append(np.nan)
            results['150_SMA'].append(np.nan)
            results['200_SMA'].append(np.nan)
            results['52W_High'].append(np.nan)
            results['Price_Above_150_SMA'].append(False)
            results['Price_Above_200_SMA'].append(False)
            results['150_SMA_Above_200_SMA'].append(False)
            results['50_SMA_Above_150_SMA'].append(False)
            results['Within_25pct_52W_High'].append(False)
            results['Meets_Trend_Template'].append(False)
            results['Volume_Spike_40pct'].append(False)
    
    # Create DataFrame
    df = pd.DataFrame(results)
    
    # Calculate RS Ranking
    df = df.replace([np.inf, -np.inf], np.nan)
    if not df['RS_Value'].dropna().empty and len(df['RS_Value'].dropna()) > 1:
        df['RS_Ranking'] = df['RS_Value'].rank(pct=True) * 100
    else:
        df['RS_Ranking'] = np.nan
        print("Warning: Insufficient valid data for ranking")
    
    # Sort by RS Ranking
    df = df.sort_values('RS_Ranking', ascending=False)
    
    # Reorder columns for clarity, including Volume_Spike_40pct
    df = df[[
        'Ticker', 'Sector', 'RS_Value', 'RS_Ranking', 'Price', 
        '50_SMA', '150_SMA', '200_SMA', '52W_High',
        'Price_Above_150_SMA', 'Price_Above_200_SMA', 
        '150_SMA_Above_200_SMA', '50_SMA_Above_150_SMA',
        'Within_25pct_52W_High', 'Meets_Trend_Template',
        'Volume_Spike_40pct'
    ]]
    
    return df

# Example usage
if __name__ == "__main__":
    # Sample list of stocks
    stock_list = [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA',
        'NVDA', 'META', 'JPM', 'V', 'WMT', 'CELH',
        'U', 'ACN', 'PLTR', 'HIMS', 'MSTR', 'SOFI', 
        'RIVN', 'TOST', 'PANW', 'CRWD', 'MU', 'RCL',
        'LLY', 'GCT', 'APP', 'SMCI', 'TTD', 'AXON', 
        'NOW', 'ANET', 'ZS', 'DECK', 'WING', 'ISRG',
        'MDB', 'MPWR', 'BABA', 'VST', 'CEG', 'NEE', 
        'NRG', 'AES', 'OKLO', 'SMR', 'SO', 'BE', 'D',
        'AEP', 'XEL', 'WEC', 'EXC', 'SRE', 'PNW','DTE', 
        'JPM', 'MA', 'GS', 'BLK', 'SPGI', 'APO', 'MTB', 
        'LPLA', 'PYPL', 'OSCR', 'RDDT', 'HOOD', 'NBIS',
        'RKLB', 'ASTS', 'UBER', 'SNOW', 'TEM', 'BB', 'TWLO',
        'AVGO', 'OKTA'
        
    ]
    
    # Calculate RS and momentum
    print("Calculating RS and Momentum Criteria with Volume Confirmation...")
    try:
        results = calculate_rs_and_momentum(stock_list)
        
        # Display full results
        print("\nRS and Momentum Results with Trend Template Components and Volume:")
        print(results.round(2))
        
        # Filter for stocks meeting Trend Template, high RS, and volume spike
        threshold = 70
        strong_stocks = results[
            (results['RS_Ranking'] >= threshold) & 
            (results['Meets_Trend_Template'] == True) &
            (results['Volume_Spike_40pct'] == True)
        ]
        print(f"\nStocks with RS Ranking >= {threshold}, Meeting Trend Template, and Volume Spike:")
        print(strong_stocks.round(2))
        
        # Save to CSV (overwrites existing file)
        csv_file = 'rs_and_momentum_with_volume.csv'
        results.to_csv(csv_file, index=False, mode='w')
        if os.path.exists(csv_file):
            print(f"\nResults overwritten to '{csv_file}'")
        else:
            print(f"\nResults saved to '{csv_file}'")
            
    except Exception as e:
        print(f"Error in calculation: {e}")

Calculating RS and Momentum Criteria with Volume Confirmation...

RS and Momentum Results with Trend Template Components and Volume:
   Ticker              Sector  RS_Value  RS_Ranking   Price  50_SMA  150_SMA  \
25    APP          Technology     24.91      100.00  415.31  361.69   225.07   
13   PLTR          Technology     19.08       98.44  101.35   84.83    55.90   
14   HIMS  Consumer Defensive     15.14       96.88   49.28   34.81    25.06   
44    SMR         Industrials     14.87       95.31   19.04   22.11    17.26   
43   OKLO           Utilities     10.77       93.75   38.79   32.60    19.50   
..    ...                 ...       ...         ...     ...     ...      ...   
42    AES           Utilities     -1.59        7.81   10.59   11.72    14.74   
33   WING   Consumer Cyclical     -1.93        6.25  234.02  292.47   344.34   
26   SMCI          Technology     -2.24        4.69   56.07   35.65    42.72   
24    GCT          Technology     -2.46        3.12   18.68   19.89